# Lab 3 - Ray and Dask

## Creating an inverted index and Word Counting

Please review Lab 2 before proceeding. Part of this lab is creating an inverted index, but using Ray instead of Parallel. We'll then move onto the more complicated word counting example.

In [2]:
import ray
ray.init(ignore_reinit_error=True)

2021-10-08 09:05:52,094	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '129.65.17.223',
 'raylet_ip_address': '129.65.17.223',
 'redis_address': '129.65.17.223:6379',
 'object_store_address': '/tmp/ray/session_2021-10-08_09-05-51_633577_8044/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-10-08_09-05-51_633577_8044/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-10-08_09-05-51_633577_8044',
 'metrics_export_port': 64467,
 'node_id': '5b85ad5b002a07e5c603f7149a7b12c125d63b69'}

In [3]:
%load_ext autoreload
%autoreload 2


# Put all your solutions into Lab1_helper.py as this script which is autograded
import Lab3_helper
    
import os
from pathlib import Path
home = str(Path.home())

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Inverted Index

**Exercise 1:**

You have already written most of what you need to use Ray to construct distributed inverted indices. Here I want you to modify Lab3_helper.py to use Ray and return the final inverted index. I'm supplying the code that divides your books into three sets.

In [5]:
group1 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group1")
group2 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group2")
group3 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group3")

In [6]:
index = Lab3_helper.merge([group1,group2,group3])

In [7]:
index['Education']

{'/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/766-0.txt': [351347]}

In [9]:
index['Education']

{'/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944],
 '/home/jupyter-pander14/csc-369-student/data/gutenberg/group3/766-0.txt': [351347]}

In [10]:
# clean up memory to help us all co-exist on the same machine
index = None
import gc
gc.collect()

86

## Word Counting
Now consider a different problem of common interest. Suppose we have a large corpus (fancy word common in natural language processing) and we want to calculate the number of times a word appears. We could try to hack our inverted index, but let's insert the requirement that this must be a clean implementation. In other words, I'll be manually reviewing your design and making you redo the assignment if it isn't "clean". 

**Exercise 2:**

Write a function that counts the words in a book. Output format shown below. You do not have to worry about punctuation and capitalization. In other words, please stick to simple f.readlines() and line.split(" "). Do not strip anything out.

In [11]:
counts = Lab3_helper.count_words(group1[0])

In [12]:
counts

{'\ufeffThe': 1,
 'Project': 64,
 'Gutenberg': 18,
 'EBook': 2,
 'of': 1518,
 'The': 311,
 'Extraordinary': 3,
 'Adventures': 3,
 'Arsene\n': 2,
 'Lupin,': 38,
 'Gentleman-Burglar,': 2,
 'by': 246,
 'Maurice': 4,
 'Leblanc\n': 4,
 '\n': 2127,
 'This': 23,
 'eBook': 5,
 'is': 392,
 'for': 246,
 'the': 3383,
 'use': 16,
 'anyone': 13,
 'anywhere': 2,
 'at': 335,
 'no': 141,
 'cost': 3,
 'and': 1254,
 'with\n': 26,
 'almost': 15,
 'restrictions': 2,
 'whatsoever.': 2,
 '': 647,
 'You': 53,
 'may': 50,
 'copy': 9,
 'it,': 50,
 'give': 28,
 'it': 340,
 'away': 14,
 'or\n': 12,
 're-use': 2,
 'under': 42,
 'terms': 21,
 'License': 8,
 'included\n': 2,
 'with': 367,
 'this': 162,
 'or': 153,
 'online': 4,
 'www.gutenberg.org\n': 2,
 'Title:': 1,
 'Arsene': 1,
 'Gentleman-Burglar\n': 1,
 'Author:': 1,
 'Posting': 1,
 'Date:': 2,
 'March': 1,
 '18,': 1,
 '2009': 1,
 '[EBook': 1,
 '#6133]\n': 1,
 'Release': 1,
 'July,': 1,
 '2004\n': 1,
 'Last': 2,
 'Updated:': 1,
 'November': 1,
 '10,': 1,
 '20

**Exercise 3**

Now let's distribute this using Ray. Please implement a function that parallelizes the word counting and subsequent merges.

In [13]:
merged_counts = Lab3_helper.merge_count_words([group1,group2,group3])

In [14]:
merged_counts['things']

3499

In [15]:
# Don't forget to push!